# Gesture Recognition for Machine Automation

## Imports

In [1]:
import numpy as np
import cv2
import os
import pyautogui
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense

## Gestures and Gesture Functions

In [2]:
gestures = np.array(["minimize", "switch", "volume_down", "volume_up"])
vids = 30
frames = 30
data = os.path.join("ges_rec_data")

In [3]:
def minimize_func():
    pyautogui.hotkey('command', 'm')

def switch_func():
    pyautogui.hotkey('command', 'tab')

def volume_down_func():
    pyautogui.press('f11')

def volume_up_func():
    pyautogui.press('f12')

## Data Collection for Features

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
def get_landmark_values(_results):
    face_vals = np.array([[val.x, val.y, val.z] for val in _results.face_landmarks.landmark]).flatten() if _results.face_landmarks else np.zeros(1404)
    left_hand_vals = np.array([[val.x, val.y, val.z] for val in _results.left_hand_landmarks.landmark]).flatten() if _results.left_hand_landmarks else np.zeros(63)
    right_hand_vals = np.array([[val.x, val.y, val.z] for val in _results.right_hand_landmarks.landmark]).flatten() if _results.right_hand_landmarks else np.zeros(63)

    return np.concatenate((face_vals, left_hand_vals, right_hand_vals))

## Engineer Feature and Label Arrays

In [37]:
gesture_dict = {gesture: pos for pos, gesture in enumerate(gestures)}
features, labels = [], []

for gesture in gestures:
        for video in range(1, vids + 1):
            all_frames = []
            for frame in range(1, frames + 1):
                np_arr = np.load(os.path.join(data, gesture, str(video), "{}.npy".format(frame)))
                all_frames.append(np_arr)
            features.append(all_frames)
            labels.append(gesture_dict[gesture])

In [38]:
X = np.array(features)
y = to_categorical(labels).astype(int)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

## Model Architecture

In [40]:
model = Sequential()
model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(30, 1530)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 30, 32)            200064    
                                                                 
 lstm_19 (LSTM)              (None, 30, 64)            24832     
                                                                 
 lstm_20 (LSTM)              (None, 128)               98816     
                                                                 
 dense_12 (Dense)            (None, 32)                4128      
                                                                 
 dense_13 (Dense)            (None, 4)                 132       
                                                                 
Total params: 327,972
Trainable params: 327,972
Non-trainable params: 0
_________________________________________________________________


## Train Model

In [41]:
epochs = 100
history = model.fit(
    X_train,
    y_train,
    epochs=epochs
)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy: {:.2f}%'.format(test_acc * 100))

Epoch 1/100
4/4 [==============================] - 2s 28ms/step - loss: 1.3814 - accuracy: 0.2745
Epoch 2/100
4/4 [==============================] - 0s 32ms/step - loss: 1.3096 - accuracy: 0.3529
Epoch 3/100
4/4 [==============================] - 0s 35ms/step - loss: 1.0376 - accuracy: 0.5196
Epoch 4/100
4/4 [==============================] - 0s 37ms/step - loss: 0.6868 - accuracy: 0.6765
Epoch 5/100
4/4 [==============================] - 0s 39ms/step - loss: 1.0368 - accuracy: 0.5686
Epoch 6/100
4/4 [==============================] - 0s 34ms/step - loss: 0.7287 - accuracy: 0.6961
Epoch 7/100
4/4 [==============================] - 0s 38ms/step - loss: 0.5696 - accuracy: 0.7451
Epoch 8/100
4/4 [==============================] - 0s 33ms/step - loss: 0.3735 - accuracy: 0.7451
Epoch 9/100
4/4 [==============================] - 0s 34ms/step - loss: 0.2208 - accuracy: 0.9118
Epoch 10/100
4/4 [==============================] - 0s 35ms/step - loss: 0.1603 - accuracy: 0.9510
Epoch 11/100
4/4 [=

In [42]:
model.save('alfred')

INFO:tensorflow:Assets written to: alfred/assets


## Load Model - ALFRED

In [43]:
model.load_weights('alfred')

2023-07-14 14:33:50.562079: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open alfred: FAILED_PRECONDITION: alfred; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


## Realtime Gesture Detection and Task Automation

In [ ]:
gesture_data = []
confidence = 0.65
previous_gesture = None

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
    landmarks = get_landmark_values(results)
    gesture_data.append(landmarks)
    gesture_data = gesture_data[-30:]

    if len(gesture_data) >= 30:
      prediction_arr = model.predict(np.expand_dims(gesture_data, axis=0))
      gesture = gestures[np.argmax(prediction_arr)]

      left_right_vals = gesture_data[0][-(63*2):] 
      zero_arr = np.zeros(63*2)

      if np.array_equal(left_right_vals, zero_arr) or np.max(prediction_arr) < confidence:
        gesture = "none"

      if gesture != previous_gesture:
        if gesture == "minimize":
          minimize_func()

        if gesture == "switch":
          switch_func()

      previous_gesture = gesture

      cv2.putText(image, '{}'.format(gesture), (800, 450), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
      cv2.imshow('MediaPipe Holistic', image)

    else:
      cv2.imshow('MediaPipe Holistic', image)

    # Exit out of webcam by pressing q
    if cv2.waitKey(1) == ord('q'):
      break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 12ms/step
